In [ ]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql

In [ ]:
#create connection to the database file 
df = pd.read_csv('smartg_new.csv')

In [ ]:
con = sql.connect('smartg.db')

In [ ]:
cur = con.cursor()

In [ ]:
cur.execute("""
CREATE TABLE smartg_data(id INTEGER PRIMARY KEY AUTOINCREMENT,customer_id TEXT, amount REAL, date TEXT)
""")
con.commit()

In [ ]:
for index, row in df.iterrows():
    cur.execute("""
    INSERT INTO smartg_data(customer_id, amount, date)
    VALUES (?,?,?)
    """,(row['spn'],row['amount'], row['date']))

con.commit()

In [ ]:
import sqlite3 as sql
import pandas as pd

class PaymentPredictor:
    def __init__(self, db_name: str):
        self.db_name = db_name

    def extract_data(self, customer_id: str):
        con = sql.connect(self.db_name)
        cur = con.cursor()

        cur.execute("""
        SELECT customer_id, amount, date
        FROM smartg_data
        WHERE customer_id = ? 
        ORDER BY date 
        """, (customer_id,))
        data = cur.fetchall()
        cur.close()
        con.close()

        # Convert data to DataFrame
        df_pd = pd.DataFrame(data, columns=['customer_id', 'amount', 'date'])
        df_pd['date'] = pd.to_datetime(df_pd['date'], errors='coerce')
        df_pd.dropna(subset=['date'], inplace=True)  # Drop rows with invalid dates

        # Aggregate the data
        data_agg = df_pd.groupby(['customer_id', 'date']).agg({'amount': 'sum'}).reset_index()
        data_agg['prev_date'] = data_agg.groupby('customer_id')['date'].shift(1)
        data_agg['payment_interval'] = (data_agg['date'] - data_agg['prev_date']).dt.days
        data_agg.dropna(subset=['payment_interval'], inplace=True)  # Drop rows with NaN intervals

        return data_agg

    def predict_next_payment(self, data_agg, window_size=5):
        # Calculate rolling mean and median intervals
        data_agg['mean_interval'] = data_agg['payment_interval'].rolling(window=window_size, min_periods=1).mean()
        data_agg['median_interval'] = data_agg['payment_interval'].rolling(window=window_size, min_periods=1).median()

        # Determine skewness
        skewness_interval = data_agg['payment_interval'].skew()
        data_agg['predicted_interval'] = data_agg.apply(
            lambda row: row['mean_interval'] if abs(skewness_interval) < 1 else row['median_interval'],
            axis=1
        )

        # Get the most recent payment date
        last_payment_date = data_agg['date'].max()

        # Select the most recent row for prediction
        last_row = data_agg.loc[data_agg['date'] == last_payment_date].iloc[0]

        # Predict next payment date
        next_payment_date = last_payment_date + pd.to_timedelta(last_row['predicted_interval'], unit='d')

        # Ensure next payment date is after the provided date
        next_payment_date = max(next_payment_date, pd.to_datetime(last_row['date']))

        # Debugging output
        print("Final Prediction Data:")
        print({
            'customer_id': last_row['customer_id'],
            'last_payment_date': last_payment_date,
            'predicted_interval': last_row['predicted_interval'],
            'next_payment_date': next_payment_date
        })

        return {
            'customer_id': last_row['customer_id'],
            'last_payment_date': last_payment_date,
            'predicted_interval': last_row['predicted_interval'],
            'next_payment_date': next_payment_date
        }

    def add_new_payment(self, customer_id: str, amount: float, date: str):
        con = sql.connect(self.db_name)
        cur = con.cursor()

        cur.execute("""
        INSERT INTO smartg_data (customer_id, amount, date)
        VALUES (?, ?, ?)
        """, (customer_id, amount, date))
        
        con.commit()
        cur.close()
        con.close()

    def get_next_payment_date(self, customer_id: str, amount: float, date: str):
        self.add_new_payment(customer_id, amount, date)
        
        data_agg = self.extract_data(customer_id)
        
        if data_agg.empty:
            return {
                'customer_id': customer_id,
                'amount': amount,
                'date': date,
                'next_payment_date': None,
            }
        
        prediction_data = self.predict_next_payment(data_agg)
        
        if prediction_data['next_payment_date'] is None:
            return {
                'customer_id': customer_id,
                'amount': amount,
                'date': date,
                'next_payment_date': None,
            }
        
        # Adjust the prediction date to be consistent with the provided date if needed
        if pd.to_datetime(date) > prediction_data['next_payment_date']:
            prediction_data['next_payment_date'] = pd.to_datetime(date) + pd.to_timedelta(prediction_data['predicted_interval'], unit='d')
        
        return {
            'customer_id': customer_id,
            'amount': amount,
            'date': date,
            'next_payment_date': prediction_data['next_payment_date'],
        }

# Example usage
predictor = PaymentPredictor('smartg.db')
customer_id = '202704338-03'
amount = 100.0
date = '2024-08-30'

result = predictor.get_next_payment_date(customer_id, amount, date)

# Display the result
print(result)